In [13]:
import glob
import os
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression
import numpy as np
import cv2
from sklearn.utils import shuffle

In [14]:
#Load Images
dataset_folder = "Dataset/"
num_classes = len(os.listdir(dataset_folder))

loadedImages = []
for filename in glob.iglob(dataset_folder + '/*/train/*', recursive=True):
    if os.path.isfile(filename): # filter dirs
        image = cv2.imread(filename)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        loadedImages.append(np.expand_dims(gray_image, 2))


In [15]:
# Create OutputVector

outputVectors = []
for i in range(len(loadedImages)):
    outputVectors.append([int(k == i // 1000) for k in range(num_classes)])


In [16]:
testImages = []

#Load Images for swing

for filename in glob.iglob(dataset_folder + '/*/test/*', recursive=True):
    if os.path.isfile(filename): # filter dirs
        image = cv2.imread(filename)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        testImages.append(np.expand_dims(gray_image, 2))

testLabels = []

for i in range(len(testImages)):
    testLabels.append([int(k == i // 100) for k in range(num_classes)])

In [19]:
# Define the CNN Model
tf.reset_default_graph()
convnet=input_data(shape=[None, 215,240,1],name='input')
convnet=conv_2d(convnet,32,2,activation='relu')
convnet=max_pool_2d(convnet,2)
convnet=conv_2d(convnet,64,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,128,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,256,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,256,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,128,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,64,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=fully_connected(convnet,1000,activation='relu')
convnet=dropout(convnet,0.75)

convnet=fully_connected(convnet,6,activation='softmax')

convnet=regression(convnet,optimizer='adam',learning_rate=0.001,loss='categorical_crossentropy',name='regression')

model=tflearn.DNN(convnet,tensorboard_verbose=0)


In [20]:
# Shuffle Training Data
loadedImages, outputVectors = shuffle(loadedImages, outputVectors, random_state=0)

# Train model
model.fit(loadedImages, outputVectors, n_epoch=50,
           validation_set = (testImages, testLabels),
           snapshot_step=100, show_metric=True, run_id='convnet_coursera')

model.save("TrainedModel/GestureRecogModel.tfl")

Training Step: 4699  | total loss: 0.00049 | time: 174.216s
| Adam | epoch: 050 | loss: 0.00049 - acc: 1.0000 -- iter: 5952/6000
Training Step: 4700  | total loss: 0.00045 | time: 180.699s
| Adam | epoch: 050 | loss: 0.00045 - acc: 1.0000 | val_loss: 0.09662 - val_acc: 0.9917 -- iter: 6000/6000
--
